# Region Boundary versus Dice Coefficient Comparison

This notebook develops an experiment to observe and compare the effects of two different approaches to evaluating a registered label region. The two evaluation approaches are as follows:
- The moving region boundary is sampled as a point set and used to construct a distribution of estimated shortest distance to the corresponding ground truth label boundary at each point.
- The dice coefficient is constructed to compare the volume of the moving and ground truth regions.

The experiment is constructed as follows:

1. Load the ground truth label image. We use the CCF v3.1 label atlas with 25mm spacing ([link](http://download.alleninstitute.org/informatics-archive/converted_mouse_ccf/annotation/ccf_2017/annotation_25.nii.gz)).
2. Select a label region at random and generate the binary image and signed distance image.
3. Initialize an ITK b-spline transform to the binary image and simulate annotation and registration error by assigning each control point a Gaussian random displacement.
4. Resample the binary image via the b-spline transform from the previous step to get the deformed binary image.
5. Record the dice coefficient between the deformed and ground truth binary images to describe the difference in volume overlap.
6. Generate a mesh at the deformed binary image region boundary.
7. Downsample the boundary mesh to a fixed maximum number of points.
8. Probe the signed distance image at each selected boundary point and record summarizing statistics to describe the boundary distance from ground truth.
9. Repeat steps 7 and 8 with different seeds for downsampling in order to more holistically describe boundary distance for a given deformed binary image.
10. Repeat steps 3 and onward with variations of b-spline control grids, control point displacement sampling distributions, and maximum point set sampling sizes.
11. Repeat steps 2 and onward with different label regions.

Results are appended to a CSV in the `notebooks\data\dice_comparison` directory after each iteration. Data plots and analysis will be made available in a subsequent notebook.

The notebook is organized as follows:
- Import modules and define methods for the experiment
- Walk through a single experimental iteration, visualized with itkwidgets
- Run iterative experiments and record results


## Import Modules and Define Methods

In [2]:
import os
import sys
import itertools

import numpy as np
import pandas as pd
import itk
import itkwidgets

sys.path.append("../scripts")
from point_distance import compute_distance_map, probe_image, describe_list

In [9]:
# Define input and output data paths
INPUT_FILENAME = r"data\CCFv31\annotation_25.nii.gz"
assert os.path.exists(INPUT_FILENAME)

RESULTS_FILENAME = "simulation-results"
RESULTS_FILEPATH = rf"data\dice_comparison\{RESULTS_FILENAME}.csv"
os.makedirs(os.path.dirname(RESULTS_FILEPATH), exist_ok=True)

In [3]:
def get_label_binary_image(label_image: itk.Image, label_value: int) -> itk.Image:
    """Construct a binary image depicting a selected region from a multi-label image"""
    return itk.binary_threshold_image_filter(
        label_image, upper_threshold=label_value, lower_threshold=label_value
    )

In [4]:
def apply_simulated_error(
    binary_image: itk.Image,
    control_grid_size: int = 5,
    displacement_scale: float = 1,
    seed: int = None,
    verbose: bool = False,
) -> (itk.Image, itk.Transform):
    """Resamples an input binary image via a b-spline transform simulating Gaussian noise displacement"""

    # Initialize the B-Spline transform control point grid
    transform = itk.BSplineTransform[itk.D, 3, 3].New()
    initializer = itk.BSplineTransformInitializer[
        type(transform), type(binary_image)
    ].New(
        transform=transform,
        image=binary_image,
        transform_domain_mesh_size=[control_grid_size] * 3,
    )
    initializer.InitializeTransform()

    if verbose:
        print(transform)

    # Apply simulated Gaussian noise to control points
    dimension = binary_image.GetImageDimension()
    for control_point_id in range(int(transform.GetNumberOfParameters() / dimension)):
        rng = np.random.default_rng(seed)
        control_point = rng.normal(size=dimension, scale=displacement_scale)
        for dim in range(dimension):
            transform.GetParameters().SetElement(
                control_point_id * dimension + dim, control_point[dim]
            )

    # Apply simulated noise transform to image
    resampled_image = itk.resample_image_filter(
        binary_image,
        transform=transform,
        reference_image=binary_image,
        use_reference_image=True,
    )

    return resampled_image, transform

In [5]:
def resample_mesh(mesh: itk.Mesh, n_samples: int, seed: int = None) -> itk.PointSet:
    """Downsamples points on a mesh boundary via uniform random picking"""
    ps = itk.PointSet[itk.F, 3].New()
    rng = np.random.default_rng(seed)
    src_indices = rng.choice(
        range(mesh.GetNumberOfPoints()), size=n_samples, replace=False
    )
    src_indices = [int(x) for x in src_indices]
    for target_index, src_index in enumerate(src_indices):
        src_pt = mesh.GetPoint(src_index)
        ps.GetPoints().InsertElement(target_index, src_pt)
    return ps

In [6]:
def compute_dice_coefficient(input_image: itk.Image, target_image: itk.Image) -> float:
    """Compute the dice coefficient to compare volume overlap between two label regions"""
    assert type(input_image) == type(resampled_image)
    dice_filter = itk.LabelOverlapMeasuresImageFilter[type(input_image)].New()
    dice_filter.SetInput(input_image)
    dice_filter.SetTargetImage(target_image)
    dice_filter.Update()
    return dice_filter.GetDiceCoefficient()

## Load Data

In [10]:
label_image = itk.imread(INPUT_FILENAME, pixel_type=itk.US)
print(type(label_image))

<class 'itk.itkImagePython.itkImageUS3'>


In [ ]:
# Use a large region for easier visualization
# 672 == Caudoputamen
# https://atlas.brain-map.org/atlas?atlas=2#atlas=2&plate=100883869&structure=672&x=7800.00029773247&y=4023.7502121343846&zoom=-3&resolution=10.21&z=6
CAUDOPUTAMEN_LABEL = 672

binary_image = get_label_binary_image(label_image, label_value=128)
print(type(binary_image))

itkwidgets.view(binary_image)

## Simulate Gaussian Error with B-splines

In [ ]:
resampled_image, noise_transform = apply_simulated_error(binary_image, verbose=True)

In [ ]:
itkwidgets.view(resampled_image)

## Sample label borders with mesh

In [ ]:
mesh_output = itk.binary_mask3_d_mesh_source(resampled_image, object_value=65535)
print(mesh_output)

In [ ]:
N_TARGET_SAMPLES = 100
downsampled_ps = resample_mesh(mesh_output, N_TARGET_SAMPLES)
print(downsampled_ps)

In [ ]:
itkwidgets.view(point_sets=downsampled_ps)

## Compute Point Distance Metric

In [ ]:
label_image = itk.imread(INPUT_FILENAME, pixel_type=itk.US)
distance_image = compute_distance_map(label_image, CAUDOPUTAMEN_LABEL)
point_distances = probe_image(distance_image, downsampled_ps)

In [ ]:
describe_list(point_distances)

## Compute DICE Coefficient

In [ ]:
print(compute_dice_coefficient(resampled_image, binary_image))

## Run Iterative Experiment

Experiment length depends on the parameters selected for iteration. Iterating over 25 regions with multiple entries and iterations for grid size, displacement, point set sample size, and seeds has been observed to require approximately eight hours on a modern desktop PC.

In [7]:
# Dataframe headers for experimental data collection
df_headers = [
    "label_value",
    "grid_size",
    "displacement_scale_mm",
    "n_point_samples",
    "sampling_seed",
    "output_dice",
    "output_mean",
    "output_stddev",
    "output_median",
]

In [11]:
# Randomly select labels to process from those available in the CCF label image
N_LABELS_TO_PROCESS = 30

label_image = itk.imread(INPUT_FILENAME, pixel_type=itk.US)
shape_label_image = itk.label_image_to_shape_label_map_filter(
    label_image, compute_perimeter=False, compute_feret_diameter=False
)

rng = np.random.default_rng()
region_indices = rng.choice(
    range(shape_label_image.GetNumberOfLabelObjects()),
    size=N_LABELS_TO_PROCESS,
    replace=False,
)

label_values = [
    shape_label_image.GetNthLabelObject(int(region_index)).GetLabel()
    for region_index in region_indices
]
print(f'Selected {", ".join([str(val) for val in label_values])}')

Selected 230, 998, 52143, 664, 393, 279, 44824, 1004, 848, 969, 9, 12, 862, 1097, 596, 132, 812, 355, 1026, 665, 466, 680, 827, 843, 204, 1084, 272, 155, 381, 757


In [ ]:
max_bspline_displacements_mm = [0.1, 0.3, 1.0, 3.0]
grid_sizes = [5, 7]
point_sample_sizes = [10, 30, 50, 75, 100, 150, 200, 250, 300]
sampling_seed_count = 5
N_ITERATIONS = 3

sample_results = pd.DataFrame(columns=df_headers)

for label_value in label_values:
    # Get the label region
    binary_image = get_label_binary_image(label_image, label_value=label_value)
    if np.max(binary_image) <= 0:
        print(
            f"Warning: No label pixels detected for label value {label_value}, skipping"
        )
        continue

    try:
        distance_image = compute_distance_map(label_image, label_value)
    except RuntimeError as e:
        print(e)
        continue

    for grid_size, displacement_scale_mm, n_points, _ in itertools.product(
        grid_sizes,
        max_bspline_displacements_mm,
        point_sample_sizes,
        range(N_ITERATIONS),
    ):
        # Warp the label region
        resampled_image, noise_transform = apply_simulated_error(
            binary_image,
            control_grid_size=grid_size,
            displacement_scale=displacement_scale_mm,
        )

        # Sample point boundaries
        mesh_output = itk.binary_mask3_d_mesh_source(
            resampled_image, object_value=int(np.max(binary_image))
        )

        for sampling_seed in range(sampling_seed_count):
            if mesh_output.GetNumberOfPoints() < n_points:
                sampling_seed = 0
                n_points = mesh_output.GetNumberOfPoints()
                downsampled_ps = mesh_output
            else:
                downsampled_ps = resample_mesh(
                    mesh_output, n_points, seed=sampling_seed
                )

            # Compute point distance metric
            point_distances = probe_image(distance_image, downsampled_ps)

            # Compute DICE coefficient
            dice_output = compute_dice_coefficient(resampled_image, binary_image)

            # Save results
            sample_result = pd.DataFrame(
                [
                    [
                        label_value,
                        grid_size,
                        displacement_scale_mm,
                        n_points,
                        sampling_seed,
                        dice_output,
                        np.mean(point_distances),
                        np.std(point_distances),
                        np.median(point_distances),
                    ]
                ],
                columns=df_headers,
            )
            print(sample_result)

            if os.path.exists(RESULTS_FILEPATH):
                sample_results = pd.read_csv(RESULTS_FILEPATH, index_col=0)
            else:
                sample_results = pd.DataFrame(columns=df_headers)

            sample_results = pd.concat(
                [sample_results, sample_result], ignore_index=True
            )
            sample_results.to_csv(RESULTS_FILEPATH)

   label_value  grid_size  displacement_scale_mm  n_point_samples  \
0        14143          5                    0.1               10   

   sampling_seed  output_dice  output_mean  output_stddev  output_median  
0              0     0.617029    -0.004403       0.022242       -0.00018  
   label_value  grid_size  displacement_scale_mm  n_point_samples  \
0        14143          5                    0.1               10   

   sampling_seed  output_dice  output_mean  output_stddev  output_median  
0              1     0.617029    -0.013225       0.023398      -0.012148  
   label_value  grid_size  displacement_scale_mm  n_point_samples  \
0        14143          5                    0.1               10   

   sampling_seed  output_dice  output_mean  output_stddev  output_median  
0              2     0.617029    -0.016338       0.023667       -0.01201  
   label_value  grid_size  displacement_scale_mm  n_point_samples  \
0        14143          5                    0.1               1

   label_value  grid_size  displacement_scale_mm  n_point_samples  \
0        14143          5                    0.1               50   

   sampling_seed  output_dice  output_mean  output_stddev  output_median  
0              0     0.543682    -0.009793       0.036707      -0.000592  
   label_value  grid_size  displacement_scale_mm  n_point_samples  \
0        14143          5                    0.1               50   

   sampling_seed  output_dice  output_mean  output_stddev  output_median  
0              1     0.543682    -0.012127       0.032141      -0.000625  
   label_value  grid_size  displacement_scale_mm  n_point_samples  \
0        14143          5                    0.1               50   

   sampling_seed  output_dice  output_mean  output_stddev  output_median  
0              2     0.543682    -0.012824       0.032145      -0.012114  
   label_value  grid_size  displacement_scale_mm  n_point_samples  \
0        14143          5                    0.1               5

   label_value  grid_size  displacement_scale_mm  n_point_samples  \
0        14143          5                    0.1              100   

   sampling_seed  output_dice  output_mean  output_stddev  output_median  
0              0     0.648619    -0.008647       0.019349      -0.000559  
   label_value  grid_size  displacement_scale_mm  n_point_samples  \
0        14143          5                    0.1              100   

   sampling_seed  output_dice  output_mean  output_stddev  output_median  
0              1     0.648619      -0.0113        0.01959      -0.000476  
   label_value  grid_size  displacement_scale_mm  n_point_samples  \
0        14143          5                    0.1              100   

   sampling_seed  output_dice  output_mean  output_stddev  output_median  
0              2     0.648619    -0.010069       0.017049      -0.000559  
   label_value  grid_size  displacement_scale_mm  n_point_samples  \
0        14143          5                    0.1              10

   label_value  grid_size  displacement_scale_mm  n_point_samples  \
0        14143          5                    0.1              200   

   sampling_seed  output_dice  output_mean  output_stddev  output_median  
0              0     0.642239     -0.00929       0.019697      -0.000535  
   label_value  grid_size  displacement_scale_mm  n_point_samples  \
0        14143          5                    0.1              200   

   sampling_seed  output_dice  output_mean  output_stddev  output_median  
0              1     0.642239    -0.007109       0.019849      -0.000559  
   label_value  grid_size  displacement_scale_mm  n_point_samples  \
0        14143          5                    0.1              200   

   sampling_seed  output_dice  output_mean  output_stddev  output_median  
0              2     0.642239    -0.008108       0.020617      -0.000625  
   label_value  grid_size  displacement_scale_mm  n_point_samples  \
0        14143          5                    0.1              20

   label_value  grid_size  displacement_scale_mm  n_point_samples  \
0        14143          5                    0.1              300   

   sampling_seed  output_dice  output_mean  output_stddev  output_median  
0              0     0.635265    -0.011874       0.023295      -0.000559  
   label_value  grid_size  displacement_scale_mm  n_point_samples  \
0        14143          5                    0.1              300   

   sampling_seed  output_dice  output_mean  output_stddev  output_median  
0              1     0.635265    -0.011996        0.02178      -0.000625  
   label_value  grid_size  displacement_scale_mm  n_point_samples  \
0        14143          5                    0.1              300   

   sampling_seed  output_dice  output_mean  output_stddev  output_median  
0              2     0.635265    -0.010408       0.021526       -0.00051  
   label_value  grid_size  displacement_scale_mm  n_point_samples  \
0        14143          5                    0.1              30

...

   label_value  grid_size  displacement_scale_mm  n_point_samples  \
0          750          5                    0.1              100   

   sampling_seed  output_dice  output_mean  output_stddev  output_median  
0              2     0.297284     -0.00734       0.020784      -0.000417  
   label_value  grid_size  displacement_scale_mm  n_point_samples  \
0          750          5                    0.1              100   

   sampling_seed  output_dice  output_mean  output_stddev  output_median  
0              3     0.297284    -0.002547       0.023788   2.753063e-09  
   label_value  grid_size  displacement_scale_mm  n_point_samples  \
0          750          5                    0.1              100   

   sampling_seed  output_dice  output_mean  output_stddev  output_median  
0              4     0.297284    -0.004219       0.022458       -0.00018  
   label_value  grid_size  displacement_scale_mm  n_point_samples  \
0          750          5                    0.1              15

   label_value  grid_size  displacement_scale_mm  n_point_samples  \
0          750          5                    0.1              200   

   sampling_seed  output_dice  output_mean  output_stddev  output_median  
0              2     0.340539    -0.007844       0.021421      -0.000592  
   label_value  grid_size  displacement_scale_mm  n_point_samples  \
0          750          5                    0.1              200   

   sampling_seed  output_dice  output_mean  output_stddev  output_median  
0              3     0.340539    -0.006383       0.021915       -0.00051  
   label_value  grid_size  displacement_scale_mm  n_point_samples  \
0          750          5                    0.1              200   

   sampling_seed  output_dice  output_mean  output_stddev  output_median  
0              4     0.340539    -0.007774       0.021311       -0.00051  
   label_value  grid_size  displacement_scale_mm  n_point_samples  \
0          750          5                    0.1              25

   label_value  grid_size  displacement_scale_mm  n_point_samples  \
0          750          5                    0.1              300   

   sampling_seed  output_dice  output_mean  output_stddev  output_median  
0              2     0.436467    -0.007619       0.018662      -0.000559  
   label_value  grid_size  displacement_scale_mm  n_point_samples  \
0          750          5                    0.1              300   

   sampling_seed  output_dice  output_mean  output_stddev  output_median  
0              3     0.436467    -0.009006       0.018244      -0.000625  
   label_value  grid_size  displacement_scale_mm  n_point_samples  \
0          750          5                    0.1              300   

   sampling_seed  output_dice  output_mean  output_stddev  output_median  
0              4     0.436467     -0.00946       0.017152      -0.000625  
   label_value  grid_size  displacement_scale_mm  n_point_samples  \
0          750          5                    0.3               1

   label_value  grid_size  displacement_scale_mm  n_point_samples  \
0          750          5                    0.3               30   

   sampling_seed  output_dice  output_mean  output_stddev  output_median  
0              2      0.09285     0.045284       0.052919       0.041462  
   label_value  grid_size  displacement_scale_mm  n_point_samples  \
0          750          5                    0.3               30   

   sampling_seed  output_dice  output_mean  output_stddev  output_median  
0              3      0.09285     0.043118       0.047137       0.028339  
   label_value  grid_size  displacement_scale_mm  n_point_samples  \
0          750          5                    0.3               30   

   sampling_seed  output_dice  output_mean  output_stddev  output_median  
0              4      0.09285      0.04381       0.042817       0.033516  
   label_value  grid_size  displacement_scale_mm  n_point_samples  \
0          750          5                    0.3               5

   label_value  grid_size  displacement_scale_mm  n_point_samples  \
0          750          5                    0.3               75   

   sampling_seed  output_dice  output_mean  output_stddev  output_median  
0              2     0.119754     0.034706       0.048851       0.023161  
   label_value  grid_size  displacement_scale_mm  n_point_samples  \
0          750          5                    0.3               75   

   sampling_seed  output_dice  output_mean  output_stddev  output_median  
0              3     0.119754     0.045641       0.051484       0.033516  
   label_value  grid_size  displacement_scale_mm  n_point_samples  \
0          750          5                    0.3               75   

   sampling_seed  output_dice  output_mean  output_stddev  output_median  
0              4     0.119754     0.036429       0.052116       0.023161  
   label_value  grid_size  displacement_scale_mm  n_point_samples  \
0          750          5                    0.3              10